In [1]:
import sys
import numpy as np
import keras
from keras import regularizers
from keras import layers
import random

if '..' not in sys.path:
    sys.path.append('..')
from car_reg_generator.car_reg_generator.uk_reg import UkRegGenerator
from car_reg_generator.car_reg_generator.uk_reg import UkRegBowVectorizer

### Data generation

In [2]:
n_train = 10000
n_test = 1000

random.seed(0)
g = UkRegGenerator()
v = UkRegBowVectorizer()

train_strs = [g.get_reg() for _ in range(n_train)]
train_vecs = np.array([v.vectorize(x) for x in train_strs])
test_strs = [g.get_reg() for _ in range(n_test)]
test_vecs = np.array([v.vectorize(x) for x in test_strs])

vec_length = len(train_vecs[0])
print(vec_length)

252


### Standard autoencoder

Design the network

In [3]:
encoding_dim = 50
input_reg = keras.Input(shape=(vec_length,))
# encoded = layers.Dense(encoding_dim, activation='relu', activity_regularizer=regularizers.l1(10e-5))(input_reg)
encoded = layers.Dense(vec_length, activation='relu')(input_reg)
encoded = layers.Dense(encoding_dim, activation='relu')(encoded)
encoded = layers.Dense(encoding_dim, activation='relu')(encoded)
encoder = keras.Model(input_reg, encoded)

encoded_input = keras.Input(shape=(encoding_dim,))
decoded = layers.Dense(encoding_dim, activation='relu')(encoded_input)
decoded = layers.Dense(vec_length, activation='relu')(decoded)
decoded = layers.Dense(vec_length, activation='sigmoid')(decoded)
decoder = keras.Model(encoded_input, decoded)

autoencoder = keras.Model(input_reg, decoder(encoder(input_reg)))

Do some learning

In [4]:
autoencoder.compile(optimizer='adam', loss='categorical_crossentropy')
autoencoder.fit(train_vecs, train_vecs,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(test_vecs, test_vecs))

Epoch 1/50
40/40 [==============================] - 1s 32ms/step - loss: 36.1173 - val_loss: 34.5494
Epoch 2/50
40/40 [==============================] - 1s 26ms/step - loss: 34.3041 - val_loss: 33.9130
Epoch 3/50
40/40 [==============================] - 1s 21ms/step - loss: 33.0613 - val_loss: 32.1408
Epoch 4/50
40/40 [==============================] - 1s 24ms/step - loss: 31.3064 - val_loss: 30.8335
Epoch 5/50
40/40 [==============================] - 1s 26ms/step - loss: 30.0407 - val_loss: 29.7868
Epoch 6/50
40/40 [==============================] - 1s 23ms/step - loss: 29.0362 - val_loss: 28.8618
Epoch 7/50
40/40 [==============================] - 1s 27ms/step - loss: 27.9557 - val_loss: 27.7067
Epoch 8/50
40/40 [==============================] - 1s 24ms/step - loss: 26.6447 - val_loss: 26.3862
Epoch 9/50
40/40 [==============================] - 1s 25ms/step - loss: 25.3434 - val_loss: 25.3786
Epoch 10/50
40/40 [==============================] - 1s 23ms/step - loss: 24.3231 - val_los

Evaluation

In [5]:
encoded_regs = encoder.predict(test_vecs)
decoded_regs = decoder.predict(encoded_regs)
print(test_strs[:10])
print([v.recover(x) for x in decoded_regs[:10]])
acc = np.sum([v.recover(x) == y for x, y in zip(decoded_regs, test_strs)]) / len(test_strs)
print('accuracy = ' + str(acc))

['AN45FVI', 'LE23DCR', 'HA99JWO', 'EQ58XKI', 'RP49FNK', 'DJ61XIM', 'WO62NON', 'EA42JCE', 'DV47KTZ', 'HM70DFU']
['AN45FVI', 'LE23DCR', 'HA99JWO', 'EQ58XKI', 'RP49FNK', 'DJ61XIM', 'WO62NON', 'EA42JCE', 'DV47KTZ', 'HM70DFU']
accuracy = 0.999
